In [1]:
# !pip install findspark

In [1]:
import findspark

findspark.init()


In [2]:
# Variables 
MIN = 60
SEC = 60
pre_trade_d = 5.5*(MIN*SEC)
trade_d = 6.5*(MIN*SEC)
post_trade_d = 4*(MIN*SEC)

total_trade_d = pre_trade_d + trade_d + post_trade_d

pre_trade_ub = pre_trade_d
trade_ub = pre_trade_ub + trade_d
post_trade_ub = trade_ub + post_trade_d


In [3]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.window import Window

spark = SparkSession.builder \
          .appName('pre_process_1min_data') \
          .getOrCreate()


Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/03/21 22:09:33 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [4]:
from pyspark.sql.types import StructType, StructField, IntegerType, TimestampType, DoubleType

schema = StructType([
    StructField("time", TimestampType()),
    StructField("open", DoubleType()),
    StructField("high", DoubleType()),
    StructField("low", DoubleType()),
    StructField("close", DoubleType()),
    StructField("volume", IntegerType())    
])


# Read csv files
df = spark.read.option("header",True).csv("./data/alpha_vantage/SPY/interval=1min/*", schema=schema)

df.printSchema()


root
 |-- time: timestamp (nullable = true)
 |-- open: double (nullable = true)
 |-- high: double (nullable = true)
 |-- low: double (nullable = true)
 |-- close: double (nullable = true)
 |-- volume: integer (nullable = true)



## Adding Time Features


In [5]:
# This column capture the time and date of SPY index price in epoch time.
df = df.withColumn("t_trade_epoch",F.unix_timestamp("time"))

# This column capture the date of SPY index price.
df = df.withColumn("t_trade_date", F.to_date("time"))

# This column capture the hour of SPY index price.
df = df.withColumn("t_trade_hour", F.hour("time"))

# This column capture the hour of SPY index price.
df = df.withColumn("t_trade_minute", F.minute("time"))


# This column capture the day of the week of SPY index price.
df = df.withColumn("t_trade_day_of_week", F.date_format("time", 'E'))

# e - epoch
# This column capture the open epoch of SPY index price per day.
df = df.withColumn("t_trade_open_e", F.min("t_trade_epoch").over(Window.partitionBy("t_trade_date")))
df = df.withColumn("t_trade_open_h", F.hour(F.min("time").over(Window.partitionBy("t_trade_date"))))

# This column capture the close epoch of SPY index price per day.
df = df.withColumn("t_trade_close_e", F.max("t_trade_epoch").over(Window.partitionBy("t_trade_date")))
df = df.withColumn("t_trade_close_h", F.hour(F.max("time").over(Window.partitionBy("t_trade_date"))))

# This column capture the number of seconds pass from trade open and trade close.
df = df.withColumn("t_time_diff_between_trade_and_open_in_sec", F.col("t_trade_epoch")-F.col("t_trade_open_e"))

# This column capture what part of the trade we are (pre trade, trade, post trade)
df = df.withColumn("t_trade_part", F.when(F.col("t_time_diff_between_trade_and_open_in_sec") < pre_trade_ub,"pre trade")
.when(F.col("t_time_diff_between_trade_and_open_in_sec") < trade_ub - 60,"trade")
.when(F.col("t_time_diff_between_trade_and_open_in_sec") <= post_trade_ub,"post trade")             
.otherwise("Unknown"))

column_list = ["t_trade_date","t_trade_part"]
win_spec = Window.partitionBy([F.col(x) for x in column_list])

# e - epoch
# This column capture the open epoch of SPY index price per trading part in day.
df = df.withColumn("t_trade_part_open_e", F.min("t_trade_epoch").over(win_spec))
df = df.withColumn("t_trade_part_open_h", F.hour(F.min("time").over(win_spec)))

# This column capture the number of seconds pass from trade part open and trade.
df = df.withColumn("t_time_diff_between_trade_and_trade_part_open_in_sec", F.col("t_trade_epoch")-F.col("t_trade_part_open_e"))

df = df.withColumn("t_precent_of_time_from_start_of_trade_pase", F.when(F.col("t_trade_part") == "pre trade", F.col("t_time_diff_between_trade_and_trade_part_open_in_sec")/pre_trade_d*100)
.when(F.col("t_trade_part") == "trade", F.col("t_time_diff_between_trade_and_trade_part_open_in_sec")/trade_d*100)
.when(F.col("t_trade_part") == "post trade", F.col("t_time_diff_between_trade_and_trade_part_open_in_sec")/post_trade_d*100)              
.otherwise(-1))

df = df.withColumn("t_precent_of_time_from_start_of_trade_day", F.col("t_time_diff_between_trade_and_open_in_sec")/total_trade_d*100)


df.createOrReplaceTempView("df")

df.printSchema()

root
 |-- time: timestamp (nullable = true)
 |-- open: double (nullable = true)
 |-- high: double (nullable = true)
 |-- low: double (nullable = true)
 |-- close: double (nullable = true)
 |-- volume: integer (nullable = true)
 |-- t_trade_epoch: long (nullable = true)
 |-- t_trade_date: date (nullable = true)
 |-- t_trade_hour: integer (nullable = true)
 |-- t_trade_minute: integer (nullable = true)
 |-- t_trade_day_of_week: string (nullable = true)
 |-- t_trade_open_e: long (nullable = true)
 |-- t_trade_open_h: integer (nullable = true)
 |-- t_trade_close_e: long (nullable = true)
 |-- t_trade_close_h: integer (nullable = true)
 |-- t_time_diff_between_trade_and_open_in_sec: long (nullable = true)
 |-- t_trade_part: string (nullable = false)
 |-- t_trade_part_open_e: long (nullable = true)
 |-- t_trade_part_open_h: integer (nullable = true)
 |-- t_time_diff_between_trade_and_trade_part_open_in_sec: long (nullable = true)
 |-- t_precent_of_time_from_start_of_trade_pase: double (nulla

In [6]:

# Test t_trade_open_e t_trade_close_e

# spark.sql("""
# select count(*) rows_c, 
#        sum(case when t_trade_open_h == 4 then 1 else 0 end) open_h,
#        sum(case when t_trade_close_h == 20 then 1 else 0 end) close_h
# from df

# """).show(100,False)

# +------+------+-------+
# |rows_c|open_h|close_h|
# +------+------+-------+
# |247708|247708|247212 |
# +------+------+-------+

# spark.sql("""
# select t_trade_open_h, t_trade_close_h, t_trade_date
# from df
# where t_trade_close_h <> 20
# group by 1,2,3
# """).show(100,False)

# +--------------+---------------+------------+
# |t_trade_open_h|t_trade_close_h|t_trade_date|
# +--------------+---------------+------------+
# |4             |17             |2022-11-25  |
# +--------------+---------------+------------+

# spark.sql("""
# select count(distinct t_trade_date) as date_c
# from df
# """).show(100,False)
# +------+
# |date_c|
# +------+
# |288   |
# +------+


In [7]:
# Test t_trade_part

# spark.sql("""

# select t_trade_date,
        
#         --time, 

#        t_trade_hour,
#        t_trade_part
# from df
# group by 1,2,3
# """).show(10000,False)

# spark.sql("""

# select count(*)
# from df
# where t_trade_part='Unknown'

# """).show(100,False)

# +--------+
# |count(1)|
# +--------+
# |0       |
# +--------+


In [8]:
# # Test t_trade_part_open_e and t_trade_part_open_h


# df.select(F.col("time"),
#        F.col("t_trade_part"),
#        F.col("t_trade_part_open_e"),
#        F.col("t_precent_of_time_from_start_of_trade_day"), 
#        F.col("t_time_diff_between_trade_and_trade_part_open_in_sec"),           
#        F.col("t_precent_of_time_from_start_of_trade_pase")).show(10000,False)

# spark.sql("""

# select t_trade_date,
        
#        time, 
#        t_trade_part,
#        t_trade_part_open_e
# from df

# """).show(10000,False)

## Adding Price Features


In [4]:
# Calculate avg_open_price_for_last_3*interval -> avg_open_price_for_last_3min
# interval = 1min

# Step 1 - convert trade time to -> trade_epoch
df = df.withColumn("trade_epoch",F.unix_timestamp("time"))

df = df.withColumn("trade_date", F.to_date("time"))

df.printSchema()
# df.show(100,False)


root
 |-- time: timestamp (nullable = true)
 |-- open: double (nullable = true)
 |-- high: double (nullable = true)
 |-- low: double (nullable = true)
 |-- close: double (nullable = true)
 |-- volume: integer (nullable = true)
 |-- trade_epoch: long (nullable = true)
 |-- trade_date: date (nullable = true)



In [19]:
df.withColumn("hour", F.hour(F.col("time")))\
.select(F.col("time"),      
         F.col("hour")).show(truncate=False)





# df.filter(df.trade_date == "2022-01-11")\
# .select(F.col("time"),        
# #         F.col("trade_date"),
#         F.col("open_trade_day_epoch_time"),
#         F.col("close_trade_day_epoch_time"),
#         F.col("time_pass_from_trade_start_in_sec")
       
#        )\
# .show(100, False)

# df.withColumn("sec_from_trade_start", F.col("close_trade_day_epoch_time")-F.col("open_trade_day_epoch_time"))\
# .select(col("firstname"),col("lastname"))
# .show(100, False)




+-------------------+----+
|time               |hour|
+-------------------+----+
|2022-10-06 20:00:00|20  |
|2022-10-06 19:59:00|19  |
|2022-10-06 19:58:00|19  |
|2022-10-06 19:57:00|19  |
|2022-10-06 19:56:00|19  |
|2022-10-06 19:55:00|19  |
|2022-10-06 19:54:00|19  |
|2022-10-06 19:53:00|19  |
|2022-10-06 19:52:00|19  |
|2022-10-06 19:51:00|19  |
|2022-10-06 19:50:00|19  |
|2022-10-06 19:49:00|19  |
|2022-10-06 19:48:00|19  |
|2022-10-06 19:47:00|19  |
|2022-10-06 19:43:00|19  |
|2022-10-06 19:42:00|19  |
|2022-10-06 19:41:00|19  |
|2022-10-06 19:40:00|19  |
|2022-10-06 19:39:00|19  |
|2022-10-06 19:38:00|19  |
+-------------------+----+
only showing top 20 rows



In [69]:

df.withColumn("rnk",F.dense_rank().over(Window.partitionBy().orderBy("trade_epoch")))\
.withColumn("avg_open_price_for_last_3min",F.avg("open").over(Window.partitionBy().orderBy("rnk").rangeBetween(-3*60,-1)))\
.withColumn("avg_open_price_for_last_5min",F.avg("open").over(Window.partitionBy().orderBy("rnk").rangeBetween(-5*60,-1)))\
.show(100, False)



23/02/21 22:29:37 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/02/21 22:29:37 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/02/21 22:29:38 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/02/21 22:29:38 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


+-------------------+------------------+------------------+------------------+------------------+------+-----------+----------+-------------------------+--------------------------+---------------------------------+---+----------------------------+----------------------------+
|time               |open              |high              |low               |close             |volume|trade_epoch|trade_date|open_trade_day_epoch_time|close_trade_day_epoch_time|time_pass_from_trade_start_in_sec|rnk|avg_open_price_for_last_3min|avg_open_price_for_last_5min|
+-------------------+------------------+------------------+------------------+------------------+------+-----------+----------+-------------------------+--------------------------+---------------------------------+---+----------------------------+----------------------------+
|2021-12-13 04:01:00|462.3392138451222 |462.652976592838  |462.3392138451222 |462.5451206483107 |2225  |1639360860 |2021-12-13|2021-12-13 04:01:00      |2021-12-13 20:00

In [8]:
#  |-- avg_open_price_for_last_3*interval e.g - 3min,  15min, 45min
#  |-- avg_open_price_for_last_5*interval e.g - 5min,  25min, 75min 
#  |-- avg_open_price_for_last_10*interva e.g - 10min, 50min, 150min

root
 |-- time: string (nullable = true)
 |-- open: string (nullable = true)
 |-- high: string (nullable = true)
 |-- low: string (nullable = true)
 |-- close: string (nullable = true)
 |-- volume: string (nullable = true)
 |-- date: date (nullable = true)



In [27]:
df.withColumn("rnk",F.dense_rank().over(Window.partitionBy().orderBy("time")))\
  .withColumn("last_week_avg",F.avg("open").over(Window.partitionBy().orderBy("rnk").rangeBetween(-7,-1)))\
  .withColumn("last_2_week_avg",F.avg("open").over(Window.partitionBy().orderBy("rnk").rangeBetween(-14,-1))              
).show()
              
              

23/02/04 21:11:02 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/02/04 21:11:02 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/02/04 21:11:02 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


+-------------------+------------------+------------------+------------------+------------------+------+---+------------------+------------------+
|               time|              open|              high|               low|             close|volume|rnk|     last_week_avg|   last_2_week_avg|
+-------------------+------------------+------------------+------------------+------------------+------+---+------------------+------------------+
|2021-12-13 04:01:00| 462.3392138451222|  462.652976592838| 462.3392138451222| 462.5451206483107|  2225|  1|              null|              null|
|2021-12-13 04:02:00|462.55492573417683|462.67258676457027|462.55492573417683| 462.6627816787041|  1078|  2| 462.3392138451222| 462.3392138451222|
|2021-12-13 04:03:00| 462.6137562493735|462.62356133523963| 462.6137562493735|462.62356133523963|  1422|  3| 462.4470697896495| 462.4470697896495|
|2021-12-13 04:04:00|  462.652976592838| 462.7608325373653|  462.652976592838| 462.7608325373653|   462|  4| 462.50263

In [19]:
week_window = Window.orderBy("Week_tmp").rangeBetween(-7, -1)

In [ ]:
df_grouped = df.groupBy(["Store_ID", "Month"]).agg(F.sum("Sales").alias("average"))

In [15]:
res_df = spark.sql("""
select time,
       open,
       high,
       low,
       close,
       volume,
       as avg_price_for_last_50_days
       
from DF_1m
where time = 'time'
group by 1
limit 100
""").show(100,False)

+----+--------+
|time|count(1)|
+----+--------+
+----+--------+

